In [341]:
import pandas as pd
import sys
!{sys.executable} -m pip install openpyxl



[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [342]:
# Load the training data 
train_data = pd.read_csv("C:/Users/cdale/OneDrive/Documents/GitHub/Movie_Webscraper/sample_data/train.csv")
train_data.drop_duplicates()
columns_to_drop = ["homepage", "overview", "poster_path", "Keywords", "tagline", "imdb_id", "belongs_to_collection", "id"]
train_data = train_data.drop(columns= columns_to_drop)

# train_data
# Explore the training data
# print(train_data.head(20))
# print(train_data.info())


In [343]:
# Filling in empty data with null values
train_data.replace('', pd.NA, inplace=True)
train_data.replace(0, pd.NA, inplace=True)

# Dropping duplicate values
train_data = train_data.drop_duplicates()

In [344]:
train_data.dropna(subset=['production_companies', 'production_countries', 'spoken_languages', 'cast', 'crew', 'budget', 'genres', 'original_language', 'popularity', 'release_date', 'runtime', 'status', 'title', 'revenue'  ], inplace=True)

# train_data


In [345]:
# Pulls the 'id' key from the dictionary and replaces that number in the "id" column. Then deletes the "belongs to collection" column.
# train_data['belongs_to_collection'] = train_data['belongs_to_collection'].apply(eval)
# train_data['id'] = train_data['belongs_to_collection'].apply(lambda x: x[0]['id'])

# Pulls the genre names from the dictionary in the "genres" column and add them to a new column "genre". Then deletes the "genres" column.
train_data['genres'] = train_data['genres'].apply(eval)
train_data['genre'] = train_data['genres'].apply(lambda x: ", ".join([genre['name'] for genre in x]))
train_data = train_data.drop(columns= "genres")


# Pulls the production company names out of the "production_companies" column and places them in the new column "production_company". Then deletes the "prodution_companies" column.
train_data['production_companies'] = train_data['production_companies'].apply(eval)
train_data['production_company'] = train_data['production_companies'].apply(lambda x: ", ".join([company['name'] for company in x]))
train_data = train_data.drop(columns= "production_companies")


# Pulls the production country names out of the "production_countries" column and places them in the new column "production_country". Then deletes the "prodution_country" column.
train_data['production_countries'] = train_data['production_countries'].apply(eval)
train_data['production_country'] = train_data['production_countries'].apply(lambda x: ", ".join([country['name'] for country in x]))
train_data = train_data.drop(columns= "production_countries")


# Pulls the spoken languages out of the "spoken_languages" column and places them in the new column "spoken_language". Then deletes the "spoken_language" column.
train_data['spoken_languages'] = train_data['spoken_languages'].apply(eval)
train_data['spoken_language'] = train_data['spoken_languages'].apply(lambda x: ", ".join([language['name'] for language in x]))
train_data = train_data.drop(columns= "spoken_languages")



# Isolates the "character" and "name"(actor) keys from the "cast" column and puts them in the new column "cast_info"
train_data['cast'] = train_data['cast'].apply(eval)
train_data['cast_info'] = train_data['cast'].apply(lambda x: [{'character': actor['character'], 'name': actor['name']} for actor in x] if isinstance(x, list) else [])
train_data = train_data.drop(columns='cast')
train_data['actors'] = train_data['cast_info'].apply(lambda x: ", ".join([actor['name'] for actor in x]))
train_data = train_data.drop(columns='cast_info')


#Isolates the 'job' and 'name' key in the crew column
train_data['crew'] = train_data['crew'].apply(eval)
def extract_director_name(crews):
    for crew in crews:
        if 'job' in crew and crew['job'] == "Director" and 'name' in crew:
            return crew['name']
    return 0
    
train_data['director'] = train_data['crew'].apply(extract_director_name)

train_data = train_data.drop(columns='crew')

train_data.replace('', pd.NA, inplace=True)
train_data.replace(0, pd.NA, inplace=True)


train_data.dropna(subset=['genre', 'production_company', 'production_country', 'spoken_language', 'actors', 'director' ], inplace=True)


# train_data




In [346]:
train_data['release_date'] = pd.to_datetime(train_data['release_date'])

train_data['month'] = train_data['release_date'].dt.month

train_data['year'] = train_data['release_date'].dt.year

train_data = train_data.drop(columns='release_date')

train_data

C:\Users\cdale\AppData\Local\Temp\ipykernel_17936\2496747735.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  train_data['release_date'] = pd.to_datetime(train_data['release_date'])


,budget,original_language,original_title,popularity,runtime,status,title,revenue,genre,production_company,production_country,spoken_language,actors,director,month,year
0,14000000,en,Hot Tub Time Machine 2,6.575393,93.0,Released,Hot Tub Time Machine 2,12314651,Comedy,"Paramount Pictures, United Artists, Metro-Gold...",United States of America,English,"Rob Corddry, Craig Robinson, Clark Duke, Adam ...",Steve Pink,2,2015
1,40000000,en,The Princess Diaries 2: Royal Engagement,8.248895,113.0,Released,The Princess Diaries 2: Royal Engagement,95149435,"Comedy, Drama, Family, Romance",Walt Disney Pictures,United States of America,English,"Anne Hathaway, Julie Andrews, H√©ctor Elizondo...",Garry Marshall,8,2004
2,3300000,en,Whiplash,64.299990,105.0,Released,Whiplash,13092000,Drama,"Bold Films, Blumhouse Productions, Right of Wa...",United States of America,English,"Miles Teller, J.K. Simmons, Melissa Benoist, A...",Damien Chazelle,10,2014
6,14000000,en,The Possession,7.286477,92.0,Released,The Possession,85446075,"Horror, Thriller","Ghost House Pictures, North Box Productions","United States of America, Canada",English,"Jeffrey Dean Morgan, Natasha Calis, Madison Da...",Ole Bornedal,8,2012
9,6000000,en,A Mighty Wind,4.672036,91.0,Released,A Mighty Wind,18750246,"Comedy, Music",Castle Rock Entertainment,United States of America,English,"Bob Balaban, Christopher Guest, John Michael H...",Christopher Guest,4,2003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2992,1135654,en,The Thief of Bagdad,3.878515,149.0,Released,The Thief of Bagdad,1213880,"Action, Adventure, Drama, Family, Fantasy",Douglas Fairbanks Pictures,United States of America,No Language,"Douglas Fairbanks, Snitz Edwards, Charles Belc...",Raoul Walsh,3,2024
2993,60000000,en,The Terminal,14.092373,128.0,Released,The Terminal,219417255,"Comedy, Drama","DreamWorks SKG, Amblin Entertainment, Parkes/M...",United States of America,"български език, English, Français, Pусский, Es...","Tom Hanks, Catherine Zeta-Jones, Stanley Tucci...",Steven Spielberg,6,2004
2997,65000000,en,The Long Kiss Goodnight,14.482345,120.0,Released,The Long Kiss Goodnight,89456761,"Crime, Action, Mystery, Thriller","New Line Cinema, Forge, The Steve Tisch Company",United States of America,English,"Geena Davis, Samuel L. Jackson, Yvonne Zima, C...",Renny Harlin,10,1996
2998,42000000,en,Along Came Polly,15.725542,90.0,Released,Along Came Polly,171963386,"Comedy, Romance","Jersey Films, Loofah Productions",United States of America,English,"Ben Stiller, Jennifer Aniston, Philip Seymour ...",John Hamburg,1,2004


In [348]:
train_data_file = "TrainData.xlsx"

train_data.to_excel(train_data_file)